In [ ]:
import logging
import math
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional, List
import json
import jsonlines
import shutil
import numpy as np
import torch
import clip
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm
from PIL import Image

#import transformers
'''
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    FlaxCLIPModel,
    CLIPModel,
    CLIPProcessor,
    CLIPTokenizer,
    TrainingArguments,
    is_tensorboard_available,
    IntervalStrategy
    
)
'''
from torchvision.datasets import VisionDataset
#from transformers.testing_utils import CaptureLogger
#from torchvision.io import ImageReadMode, read_image
'''
from torchvision.transforms import (
    # added for image augmentation
    ToPILImage,
    RandomCrop,
    ColorJitter,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    ToTensor,
    # /added for image augmentation
    CenterCrop, 
    ConvertImageDtype, 
    Normalize, 
    Resize
)
'''
#from torchvision.transforms.functional import InterpolationMode

import jsonlines
from pathlib import Path
from typing import Optional, Callable
from importlib.util import find_spec
import torch.nn as nn
logger = logging.getLogger(__name__)

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the model weights should be initialized and trained. Choose one of `[float32, float16, bfloat16]`."
        },
    )
    save_optimizer: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to store full train state including optimizer."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    data_dir: Optional[str] = field(
        default=None, metadata={"help": "Path to local folder containing data files."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file (a jsonlines file)."},
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    text_column_name: Optional[str] = field(
            default='text',
            metadata={"help": "Column containing main text data."},
    )
    augment_images: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    augment_captions: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    captions_per_image: Optional[int] = field(
        default=5,
        metadata={"help": "Number of captions per image to use when creating train dataset."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`validation_file` should be a csv, a json or a txt file."


@dataclass
class ImageAugmentationArguments:
    """
    Arguments for image augmentations configuration
    """
    random_horizontal_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random horizontal flip" }
    )
    random_vertical_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random vartical flip" }
    )


In [ ]:
data_args = DataTrainingArguments(
    dataset_name = None,
    data_dir = "./data",
    train_file = "data/train_lines.jsonl",
    validation_file = "data/train_lines.jsonl",
    max_train_samples = None,
    max_eval_samples = None,
    overwrite_cache = False,
    validation_split_percentage = 5,
    preprocessing_num_workers = 0,
    augment_images = False, ##Can be changed to augment captions and images (NEEDS FUTURE WORK)
    augment_captions = False,
    captions_per_image = 5,
)

augmentation_args = ImageAugmentationArguments()

In [ ]:
class ImageTextDataset(VisionDataset):
    """
    Dtaset for loading image-text data for tasks like CLIP training, Image Captioning.

    Args:
        root: (string): The root path where the dataset is stored.
            The expected format is jsonlines where each line is a json object containing to keys.
            `filename`: The path to the image.
            `captions`: An `array` of captions.
        split: (string): Dataset split name. Is used for parsing jsonl files from `root` folder.
        captions_per_image: (int): number of captions per image to use. Defaults to 5.
        augment_captions: (bool): If true the jsonl files with `textaug_` prefix are selected from root
            folder. 
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        split: str, 
        captions_per_image:int = 5,
    ):
        super().__init__(root, None, None, None)
        self.root = root
        prefix = ""
        filepaths = list(Path(root).glob(f"{prefix}train*.jsonl"))
        fps_empty_msg = f"""\
        The `filepaths` is empty. Please make sure that `root` folder contains jsonl files
        named properly: [textaug_]{split}*.jsonl.
        `textaug_` prefix is expected if `augment_captions` is `True`.
        """
        assert len(filepaths) > 0, fps_empty_msg
        
        self.captions = []
        self.image_paths = []
        for count, filepath in enumerate(filepaths):
            with jsonlines.open(filepath, "r") as reader:
                length = len(list(jsonlines.open(filepath, "r")))
                step =0 
                for example in reader:
                    if captions_per_image ==1:
                        captions_per_image_cur = 1
                    else:
                        captions_per_image_cur = len(example["Captions"])
                    
                    if split=='train' and step < 0.85*length:
                        self.captions.extend(clip.tokenize(example["Captions"][:captions_per_image_cur]))
                        self.image_paths.extend([example["URL"]] * captions_per_image_cur)
                    elif split=='valid' and step > 0.85*length:
                        self.captions.extend(clip.tokenize(example["Captions"][:captions_per_image_cur]))
                        self.image_paths.extend([example["URL"]] * captions_per_image_cur)
                    step+=1
        print(f"{count+1} input files for {split} split found")
    
    def _load_image(self, idx: int):
        path = f"{self.root}/image_dataset/train_images/{self.image_paths[idx]}"
        return preprocess(Image.open(path))

    def _load_target(self, idx):
        return self.captions[idx]

    def __getitem__(self, index: int):
        path = f"{self.root}/image_dataset/train_images/{self.image_paths[index]}"
        image = preprocess(Image.open(path))
        target = self.captions[index]

        if self.transforms is not None:
            image, target = self.transforms(image, target)
        return image, target

    def __len__(self) -> int:
        return len(self.captions)

In [ ]:
torch.cuda.is_available()
device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.

In [ ]:
device

In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training


In [ ]:
train_dataset = ImageTextDataset(
    data_args.data_dir,
    'train',
    captions_per_image=data_args.captions_per_image,
)

eval_dataset = ImageTextDataset(
    data_args.data_dir,
    'valid',
    captions_per_image=1,
)


In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

checkpoint = torch.load("model_checkpoint/model_1_epoch2.pt")
'''
# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 
'''
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-6, betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

In [ ]:
import gc


In [ ]:
# use your own data
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size = 16, shuffle=True) #Define your own dataloader
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,batch_size = 16, shuffle=True) #Define your own dataloader
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float()
        
#optimizer = torch.optim.Adam(model.parameters(), lr=5e-7,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset
clip.model.convert_weights(model)
#model.float()
epoch_avg =0 
cost_hist = []
eval_epoch_avg =0 
eval_cost_hist = []

total_loss =0 
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

def cross_entropy(logits, axis):
    logprobs = nn.functional.log_softmax(logits, dim=axis)
    nll = torch.diag(logprobs)
    ce = -torch.mean(nll)
    return ce

def clip_loss(similarity):
    loss = (cross_entropy(similarity, axis=0) + cross_entropy(similarity, axis=1)) / 2
    return loss


epochs = tqdm(range(5))
# add your own code to track the training progress.
for epoch in epochs:
    steps_per_epoch = len(train_dataset) // 16
    epoch_avg =0 
    # train
    steps_trained_progress_bar = tqdm(range(steps_per_epoch), desc=f"Training. Loss: {total_loss}", position=1, leave=False)
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        steps_trained_progress_bar.update(1)
        images,texts = batch 

        images= images.to(device)
        texts = texts.to(device)

        logits_per_image, logits_per_text = model(images, texts)
        #logits = model(images, texts)[0]

        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        #total_loss = clip_loss(logits)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        #print(total_loss , alt_loss)
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)
        
        epoch_avg+= total_loss.item()
        steps_trained_progress_bar.set_description('\rStep: %s, Total loss%.4f' % (step, total_loss))
        
        
    # EVAL
    eval_steps_per_epoch = len(eval_dataset) // 16
    eval_epoch_avg =0 

    eval_steps_trained_progress_bar = tqdm(range(eval_steps_per_epoch), desc=f"Eval. Loss: {total_loss}", position=1, leave=False)
    for step, batch in enumerate(eval_dataloader):
        eval_steps_trained_progress_bar.update(1)
        images,texts = batch 

        images= images.to(device)
        texts = texts.to(device)

        #logits_per_image, logits_per_text = model(images, texts)
        logits = model(images, texts)[0]
        #ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = clip_loss(logits).item()
        eval_epoch_avg+=total_loss
        eval_steps_trained_progress_bar.set_description('\rStep: %s, Total loss%.4f' % (step, total_loss))
    epoch_avg /= steps_per_epoch 
    eval_epoch_avg /= eval_steps_per_epoch
    eval_cost_hist.append(eval_epoch_avg)
    cost_hist.append(epoch_avg)
    
    epochs.set_description('\rEpoch: %s, Training Loss: %.4f, Validation Loss: %.4f' % (epoch, epoch_avg, eval_epoch_avg))
    torch.save({
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_1_epoch{epoch}.pt") #just change to your preferred folder/filename

In [ ]:
import gc
gc.collect()

In [ ]:
device= 'cuda:0'

In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

In [ ]:
checkpoint = torch.load("model_checkpoint/model_1_epoch19.pt")
'''
# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 
'''
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
image = preprocess(Image.open('data/image_dataset/test_images/3625376_1dfc183f7b_z.jpg')).unsqueeze(0).to(device)
text = clip.tokenize(["a black and white dog holding a ball near sea"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
for l, p in zip(["A black dog","a white and black dog ","a black cat","a black and white dog near sea","a black and white dog holding a ball near sea"], probs[0]):
    print(f"{l:<16} {p:.4f}")

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
similarity

In [ ]:
logit_scale = model_a.logit_scale.exp()

In [ ]:
logit_scale

In [ ]:
plt.imshow(Image.open('data/image_dataset/test_images/3625376_1dfc183f7b_z.jpg'))

In [ ]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_1_epoch{epoch}.pt") #just change to your preferred folder/filename